In [1]:
import numpy as np
import pandas as pd
import sqlite3

The goal of this notebook is to import US wildfire data from https://www.kaggle.com/rtatman/188-million-us-wildfires and analyze if the wildfires of recent years are normal or abnormal.

Plot the number of wildfires and the total acreage of wildfires over time
Plot for California alone
Plot for entire US

Further Analysis
Plot wildfires on US map

In [2]:
# Import database and load into appropriate dataframes
wf_data = sqlite3.connect("FPA_FOD_20170508.sqlite")
df = pd.read_sql_query("select OBJECTID,FOD_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY from Fires", 
                       wf_data) #remove limit for final evaluation

In [3]:
# create separate dataframe for only california
ca_df=df[df['STATE']=='CA']

Two DataFrames created

    df: data from all US fires
    ca_df: data from all California fires
    

In [4]:
year_df = df.groupby('FIRE_YEAR')
year_ca_df = ca_df.groupby('FIRE_YEAR')

In [5]:
year_df.count()['FOD_ID']

FIRE_YEAR
1992     67975
1993     61989
1994     75955
1995     71472
1996     75574
1997     61450
1998     68370
1999     89363
2000     96416
2001     86587
2002     75656
2003     68261
2004     69279
2005     88604
2006    114004
2007     95573
2008     85378
2009     78325
2010     79889
2011     90552
2012     72769
2013     64780
2014     67753
2015     74491
Name: FOD_ID, dtype: int64

In [7]:
year_ca_df.count()['FOD_ID']

FIRE_YEAR
1992    10833
1993     8270
1994     8654
1995     7381
1996     9173
1997     7930
1998     6864
1999     8910
2000     6977
2001     8182
2002     7133
2003     7913
2004     7418
2005     6678
2006     8271
2007    10151
2008     7744
2009     6940
2010     5776
2011     8562
2012     7208
2013     8723
2014     6494
2015     7365
Name: FOD_ID, dtype: int64

In [13]:
ca_df.head(100)

,OBJECTID,FOD_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,1,1,FOUNTAIN,2005,2453403.5,33,1300,0.10,A,40.036944,-121.005833,CA,63
1,2,2,PIGEON,2004,2453137.5,133,0845,0.25,A,38.933056,-120.404444,CA,61
2,3,3,SLACK,2004,2453156.5,152,1921,0.10,A,38.984167,-120.735556,CA,17
3,4,4,DEER,2004,2453184.5,180,1600,0.10,A,38.559167,-119.913333,CA,3
4,5,5,STEVENOT,2004,2453184.5,180,1600,0.10,A,38.559167,-119.933056,CA,3
5,6,6,HIDDEN,2004,2453186.5,182,1800,0.10,A,38.635278,-120.103611,CA,5
6,7,7,FORK,2004,2453187.5,183,1800,0.10,A,38.688333,-120.153333,CA,17
7,8,8,SLATE,2005,2453437.5,67,1300,0.80,B,40.968056,-122.433889,CA,None
8,9,9,SHASTA,2005,2453444.5,74,1200,1.00,B,41.233611,-122.283333,CA,None
9,10,10,TANGLEFOOT,2004,2453187.5,183,1800,0.10,A,38.548333,-120.149167,CA,5


In [28]:
county_ca_df = ca_df.groupby('COUNTY')

In [34]:
series_county_counts = county_ca_df.count()['FOD_ID']

In [36]:
series_county_counts.sort_values(ascending=True)

COUNTY
TEHAMA                     1
Orange County              1
Orange / Riverside         1
Orange                     1
067                        1
Monterey County            1
Riverside County           1
Marin                      1
Trinity County             1
Madera County              1
99                         1
RIVERSIDE, SAN BRDNO       1
MONO                       1
LAC                        1
SACRAMENTO/AMADOR          1
SAN BERNADINO              1
Ventura                    1
Ventura County             1
Santa Clara (See 44)       1
Humbolt                    1
Santa Barbara County       1
Douglas                    1
KERN                       1
LAKE - SONOMA              1
Riverside                  1
Humboldt County            2
Kern County                2
SIERRA                     3
San Diego County           5
Sacramento                 6
                        ... 
Nevada                   574
Siskiyou                 578
MENDOCINO                583
Butte  

In [ ]:
ca_df